In [ ]:
import os
import gc
import sys
import json
import time
import math
import random
from datetime import datetime
from collections import Counter, defaultdict
from sklearn.metrics import mean_absolute_error, roc_auc_score

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from tqdm.auto import tqdm

from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

import warnings
warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
PATH = '../input/tabular-playground-series-nov-2021'
train = pd.read_csv(os.path.join(PATH, 'train.csv'))
test = pd.read_csv(os.path.join(PATH, 'test.csv'))
sub = pd.read_csv(os.path.join(PATH, 'sample_submission.csv'))

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    competition='TPS_NOV_21_V1'
    _wandb_kernel='santosh'
    print_freq=1000
    num_workers=4
    model_name='LogisticRegression'
    scheduler='CosineAnnealingLR' # ['linear', 'cosine', 'ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    batch_scheduler=False
    #num_warmup_steps=100 # ['linear', 'cosine']
    #num_cycles=0.5 # 'cosine'
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=10 # CosineAnnealingLR
    #T_0=50 # CosineAnnealingWarmRestarts
    epochs=30
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=64
    lr=1e-3
    min_lr=1e-6
    weight_decay=1e-6
    batch_size=128
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    inference=True

In [ ]:
pseudo_75_index = np.load('../input/pseudo-data/test_0_75.npy')
pseudo_35_index = np.load('../input/pseudo-data/test_0_35.npy')

In [ ]:
test.loc[pseudo_75_index, 'target'] = 1

In [ ]:
test.loc[pseudo_35_index, 'target'] = 0

In [ ]:
index = np.concatenate([pseudo_75_index , pseudo_35_index])

In [ ]:
train = pd.concat([train , test.loc[index]])

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.target.value_counts()

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony = None
except:
    #anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

    
def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

run = wandb.init(project="TPS_NOV_21_V1", 
                 name=CFG.model_name,
                 config=class2dict(CFG),
                 group=CFG.model_name,
                 job_type="train",
                 anonymous=anony)



In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything()

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)    

for n, (train_index, val_index) in enumerate(kf.split(train, train['target'])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
print(train.groupby('fold').size())

In [ ]:
columns = [col for col in train.columns if col not in ['id', 'target', 'fold'] ]

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.features = df[columns].values
        self.targets = df[['target']].values
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float)
        targets = torch.tensor(self.targets[idx], dtype=torch.float)
        return features, targets
    

class TestDataset(Dataset):
    def __init__(self, df):
        self.features = df[columns].values
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float)
        return features

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = nn.Sequential(
            nn.BatchNorm1d(len(columns)),
            nn.Linear(len(columns), 1024),
            nn.BatchNorm1d(1024),
            nn.SELU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.SELU(),
            nn.Dropout(0.2),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.SELU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

        
    def forward(self, features):
        bs = features.size(0)
        x = self.layer(features).view(bs, -1)
        return x

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    losses = AverageMeter()
    for step, (feature, y) in enumerate(train_loader):
        feature, y = feature.to(device), y.to(device)
        batch_size = feature.size(0)
        pred = model(feature)
        loss = criterion(pred, y) 
        losses.update(loss.item(), batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
            loss.backward()            
        else:
            loss.backward()
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            if CFG.batch_scheduler:
                scheduler.step()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'LR: {lr:.6f}  '
                  .format(
                   epoch+1, step, len(train_loader),
                   loss=losses,
                   lr=scheduler.get_lr()[0],
                   ))
        wandb.log({f"[fold{fold}] loss": losses.val,
                   f"[fold{fold}] lr": scheduler.get_lr()[0]})
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    preds = []
    losses = AverageMeter()
    for step, (feature, y) in enumerate(valid_loader):
        feature, y = feature.to(device), y.to(device)
        batch_size = feature.size(0)
        with torch.no_grad():
            pred = model(feature)
        loss = criterion(pred, y) 
        losses.update(loss.item(), batch_size)
        preds.append(pred.view(-1).detach().cpu().numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(
                   step, len(valid_loader),
                   loss=losses,
                   ))
    preds = np.concatenate(preds)
    return losses.avg, preds


def inference_fn(test_loader, model, device):
    model.eval()
    model.to(device)
    preds = []
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    for step, (feature) in tk0:
        feature = feature.to(device)
        with torch.no_grad():
            pred = model(feature)
        preds.append(pred.view(-1).detach().cpu().numpy())
    preds = np.concatenate(preds)
    return preds

In [ ]:
def train_loop(folds, fold):

    trn_idx = folds[folds['fold'] != fold].index
    val_idx = folds[folds['fold'] == fold].index
    
    train_folds = train.loc[trn_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    y_true = valid_folds['target'].values
    
    
    rs = preprocessing.RobustScaler()
    train_folds[columns] = rs.fit_transform(train_folds[columns])
    valid_folds[columns] = rs.transform(valid_folds[columns])
    
    
    test[columns] = rs.transform(test[columns])
   
    train_dataset = TrainDataset(train_folds)
    valid_dataset = TrainDataset(valid_folds)
    
    
    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    model = CustomModel()
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    
    def get_scheduler(optimizer):
        if CFG.scheduler=='linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif CFG.scheduler=='cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=CFG.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=CFG.num_cycles
            )
        elif CFG.scheduler=='ReduceLROnPlateau':
            scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=CFG.factor, patience=CFG.patience, verbose=True, eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=CFG.T_0, T_mult=1, eta_min=CFG.min_lr, last_epoch=-1)
        return scheduler

    scheduler = get_scheduler(optimizer)


    criterion = nn.BCEWithLogitsLoss()

    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
        
        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, CosineAnnealingWarmRestarts):
            scheduler.step()

        
        score = roc_auc_score(y_true, preds)
        
        elapsed = time.time() - start_time

        wandb.log({f"[fold{fold}] epoch": epoch+1, 
                   f"[fold{fold}] avg_train_loss": avg_loss, 
                   f"[fold{fold}] avg_val_loss": avg_val_loss,
                   f"[fold{fold}] score": score})
        
        if score < best_score:
            best_score = score
            torch.save({'model': model.state_dict(),
                        'preds': preds},
                        f"fold{fold}_best.pth")
            
    preds = torch.load(f"fold{fold}_best.pth", map_location=torch.device('cpu'))['preds']
    valid_folds['preds'] = preds

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [ ]:
def main():
        
    def get_result(result_df):
        preds = result_df['preds'].values
        labels = result_df['target'].values
        score = roc_auc_score(labels, preds)
    
    if CFG.train:
        # train 
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                get_result(_oof_df)
        # CV result
        get_result(oof_df)
        # save result
        oof_df.to_csv('oof_df.csv', index=False)
    
    if CFG.inference:
        test_dataset = TestDataset(test)
        test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size * 2, shuffle=False, num_workers=CFG.num_workers, pin_memory=True)
        for fold in CFG.trn_fold:
            model = CustomModel()
            path = f"fold{fold}_best.pth"
            state = torch.load(path, map_location=torch.device('cpu'))
            model.load_state_dict(state['model'])
            predictions = inference_fn(test_loader, model, device)
            test[f'fold{fold}'] = predictions
            del state, predictions; gc.collect()
            torch.cuda.empty_cache()
        # submission
        test['target'] = test[[f'fold{fold}' for fold in range(CFG.n_fold)]].mean(1)
        test[['id','target']+[f'fold{fold}' for fold in range(CFG.n_fold)]].to_csv('raw_submission.csv', index=False)
        test[['id','target']].to_csv('submission.csv', index=False)
    
    wandb.finish()

In [ ]:
if __name__ == '__main__':
    main()